In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from datetime import datetime
from matplotlib import pyplot as plt
from collections import defaultdict
import json
import os


month_dict = { "Ocak":1, "Şubat":2, "Mart":3, "Nisan":4, "Mayıs":5, "Haziran":6, "Temmuz":7, "Ağustos":8, "Eylül":9, "Ekim":10, "Kasım":11, "Aralık":12}


/Users/kaanguraysirin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
LIST_PATH= "list.txt"
DATA_PATH = "data_1.json"

In [3]:
def convert_date(date):
    date= date.split(",")[1].strip()
    #months are in Turkish
    for key in month_dict:
        date = date.replace(key, str(month_dict[key]))
    date = date.split(" ")
    date = date[2]+"-"+date[1]+"-"+date[0]
    date = datetime.strptime(date, "%Y-%m-%d")
    return date

def convert_date2(date):
    date= date.split(",")[1].strip()
    #months are in Turkish
    for key in month_dict:
        date = date.replace(key, str(month_dict[key]))
    date = date.split(" ")
    date = date[0]+"."+date[1]+"."+date[2]
    return str(date)


def get_article_id(query, date_list):
    url="https://tr.wikipedia.org/w/index.php?title={}&action=history&limit=5000".format(query.replace(" ", "_"))
    resp=requests.get(url)
    soup = bs(resp.text, "html.parser")
    articles= soup.find_all("ul", {"class":"mw-contributions-list"})
    datelist= date_list.copy()

    ret_map = defaultdict(list)

    while True:
        for art in articles:    
            #if a date is smaller than the given date, return the id
            #since the articles are sorted returned value will be the closest date to the given date 
            if datelist[0] >= convert_date(art.find("a", {"class":"mw-changeslist-date"}).text):
                #print("Date is:",convert_date(art.find("a", {"class":"mw-changeslist-date"}).text))
                ret_map[query].append([art.find("li")["data-mw-revid"], convert_date2(art.find("a", {"class":"mw-changeslist-date"}).text)])
                datelist.pop(0)

                if len(datelist)==0:
                    return ret_map

        try: #if the date is not found in the history, check the next page
            next_page = soup.find("a", {"class":"mw-nextlink"})
            next_page = next_page["href"]
            resp=requests.get("https://tr.wikipedia.org"+next_page)
            soup = bs(resp.text, "html.parser")
            articles= soup.find_all("ul", {"class":"mw-contributions-list"})
        except: #if there is no more pages to check, return None
            print(f"No data for below {str(datelist[0].date()) } for {query}")
            break
        
    return ret_map

def get_wiki_page(pid): 
    # get the article with the given id
    url="https://tr.wikipedia.org/w/index.php?oldid={}".format(pid)
    resp=requests.get(url)
    soup = bs(resp.text, "html.parser")
    article = soup.find("div", {"id":"mw-content-text"})
    return article.text


def get_history_data(query, date):
    pid = get_article_id(query, date)
    return pid
    article = get_wiki_page(pid)
    return article
    


In [4]:
#usage Query; Date year, month, day
date_list= [datetime(i,1,1) for i in range(2025,2015,-1)]

In [5]:
my_article= get_history_data("Ekrem İmamoğlu", date_list)


No data for below 2019-01-01 for Ekrem İmamoğlu


In [6]:
print(len(my_article["Ekrem İmamoğlu"]))

6


In [7]:
if not os.path.exists(LIST_PATH):
    with open(LIST_PATH, "w") as f:
        f.write("\t".join(["ID", "Query", "Date"])+ "\n")

else:
    data= pd.read_csv(LIST_PATH, sep="\t")
    if data.shape[0]==0:
        id_list=[]
    else:
        id_list= data["ID"].tolist()

In [8]:
with open(LIST_PATH, "a") as f:
    for key in my_article:
        for val in my_article[key]:
            if int(val[0]) not in id_list:
                f.write("\t".join([val[0], key.replace(" ","_") , val[1]])+ "\n")
                id_list.append(int(val[0]))

In [9]:
df = pd.read_csv(LIST_PATH, sep="\t")
data = {}

if not os.path.exists(DATA_PATH):
    with open(DATA_PATH, "w") as f:
        pass

else:
    if os.path.getsize(DATA_PATH)!=0:
        with open(DATA_PATH, "r") as f:
            data= json.load(f)



with open(DATA_PATH, "w") as f:
    for i in range(df.shape[0]):
        #if df["ID"][i] not in json_data:
            data[df["ID"][i]] = get_wiki_page(df["ID"][i]).replace("\n", " ").strip()

    #convert keys to string
    data = {str(key):val for key, val in data.items()}
    json.dump(data, f, ensure_ascii=False)

    